In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["font.sans-serif"] = "Helvetica"
import bokeh.palettes
from neuprint import Client, fetch_simple_connections, fetch_adjacencies, NeuronCriteria as NC, SynapseCriteria as SC

# Load the authentication token 
auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:v1.2.1', token=auth_token)
except:
    np_client = None

from get_connectome import get_connectome, connectome_to_undirected

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
# pull connectome for ovien
oviEN = 452689494
ovien_connectome = get_connectome(oviEN, exclude_main_neurons=True, connectome_scope='input')
ovien_connectome

/Users/rhessa/miniconda3/envs/flybrainlab/lib/python3.11/site-packages/neuprint/client.py:609: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]


  0%|          | 0/3 [00:00<?, ?it/s]

,bodyId_pre,bodyId_post,weight
0,267551639,297921608,24
1,267551639,297947227,2
2,267551639,298254517,5
3,267551639,298258714,2
4,267551639,299289811,5
...,...,...,...
26522,5901213494,5813010210,1
26523,5901213494,5813010282,1
26524,5901213494,5813020840,1
26525,5901213494,5813055903,1


In [3]:
# undirected list
ovien_connectome_undirected = connectome_to_undirected(ovien_connectome)
ovien_connectome_undirected

,source,target,weight
0,267551639,297921608,41
1,267551639,297947227,19
2,267551639,298254517,12
3,267551639,298258714,4
4,267551639,299289811,5
...,...,...,...
20922,5901213494,5813010210,1
20923,5901213494,5813010282,1
20924,5901213494,5813020840,1
20925,5901213494,5813055903,1


In [4]:
ovien_connectome_undirected.to_csv('ovien_connectome_undirected.txt', header=False, index=False, sep=',')

In [5]:
# import
clustering = pd.read_csv('~/Downloads/clustering_result_ovien.txt', sep=' ', header=None, names=['id', '0.0'])
clustering

,id,0.0
0,1010947498,1
1,1041666949,1
2,1042656011,1
3,1043289707,2
4,1072063538,3
...,...,...
464,924049792,3
465,948851462,4
466,949534412,4
467,950229431,4


In [6]:
# fetch adjacencies
input_neuron_info, input_connection_info = fetch_adjacencies(NC(status='Traced', cropped=False), oviEN)
input_neuron_info

,bodyId,type,instance
0,452689494,SMP550,SMP550_R
1,267551639,pC1c,pC1c_R
2,294436967,PPL203,PPL203_R
3,296203324,SMP363,SMP363_R
4,296531366,SMP362,SMP362_R
...,...,...,...
465,5813087438,LHAV6b2_a,LHAV6b2_a_R
466,5813108129,SMP358,SMP358_R
467,5813133640,None,(MBDLaxon2)
468,5901203505,SLP212,SLP212_R


In [7]:
# find ids in input neurons and not in clustering
inputs = set(input_neuron_info['bodyId'])
clustering_ids = set(clustering['id'])

inputs_not_in_clustering = inputs - clustering_ids
inputs_not_in_clustering



{452689494}

In [8]:
# Find neurons in input_neuron_info dataframe
input_neuron_info[input_neuron_info['bodyId'].isin(inputs_not_in_clustering)]


,bodyId,type,instance
0,452689494,SMP550,SMP550_R
